Anything that is recorded repeatedly at many points in time forms a time series.

Many time series are fixed frequency - data points occur at regular intervals according to some rules.

How to mark or refer time series data:

- Timestamps: specific instants in time
- Fixed period: Such as whole month or whole year
- Intervals of time: Indicated by a start and end timestamp. Periods can be thought of as special cases of intervals.
- Experiment or elapsed time: Each timestamp is a measure if time related to a particular start time. (The index may be an integer or floating-point number indicating elapsed time from the start of the experiment)


In [2]:
import numpy as np
import pandas as pd


# 11.1 Date and Time Data Types and Tools


In [3]:
from datetime import datetime, timedelta

now = datetime.now()
now
now.year, now.month, now.day

# datetime stores both the date and time down to the microsecond.
# datetime.timedelta represents the temporal difference between two datetime objects

delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta
delta.days, delta.seconds

# Add or substract a timedelta or multipole to a datetime object to yield a new shifted object
start = datetime(2011, 1, 7)
start + timedelta(12)
start - 2 * timedelta(12)


datetime.datetime(2010, 12, 14, 0, 0)

## Converting Between String and Datetime


In [ ]:
# Format datetime objects and pandas Timestamp objects as strings using str or strftime method

stamp = datetime(2011, 1, 3)
str(stamp)
stamp.strftime("%Y-%m-%d")

# Use same format code convert strings to date
value = "2011-01-03"

# use datetime.strptime to parse a date with a known format
datetime.strptime(value, "%Y-%m-%d")
datestrs = ["7/6/2011", "8/6/2011"]
[datetime.strptime(x, "%m/%d/%Y") for x in datestrs]


## Pandas datetime object

use to_datetime to parse string to date time.
datetime objects also have a number of local-specific formatting options for systems in other countries or languages.


In [ ]:
# pandas.to_datetime method parses different kinds of date representations, e.g. ISO8601 formats
datestrs = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]

pd.to_datetime(datestrs)


# NaT(Not a Time) is pandas's null value for timestamp data
idx = pd.to_datetime(datestrs + [None])
idx


# 11.2 Time Series Basics

A basic kind of time series object in pandas is a Series indexed by timestamps, which is often represented outside of pandas as Python strings or datetime objects


In [ ]:
dates = [
    datetime(2011, 1, 2),
    datetime(2011, 1, 5),
    datetime(2011, 1, 7),
    datetime(2011, 1, 8),
    datetime(2011, 1, 10),
    datetime(2011, 1, 12),
]

ts = pd.Series(np.random.standard_normal(6), index=dates)

ts


In [ ]:
# perform arithmetic operation between differently indexed time series.
# [::2] selects every second elements in ts
ts + ts[::2]


In [ ]:
# Find the index data type
ts.index.dtype


In [ ]:
ts.index


In [ ]:
# scalar values from a datetimeIndex are pandas timestamp objects

stamp = ts.index[0]
stamp


A pandas.Timestamp can be substituted most places where your would use a datetime object. But reverse it not true.
pandas.Timestamp can store nano seconds and frequency information, where datetime object cannot.


## Indexing, Selection, Subsetting

Time series behaves like any other Series when you are indexing and selecting data based on the label.


In [ ]:
# Select by index
stamp = ts.index[2]

ts[stamp]


In [ ]:
# Select by string that is interpretable as a date
# Both of these 3 works
ts["2011/01/10"]
ts["20110110"]
ts["2011-01-10"]


In [ ]:
# Pace a year or a month to select slices of data
longer_ts = pd.Series(
    np.random.standard_normal(1000), index=pd.date_range("2011-01-01", periods=1000)
)


In [ ]:
longer_ts["2012"]

# Select the month
longer_ts["2012-02"]

# Select by datetime object
ts[datetime(2011, 1, 7)]

# If time stamps not contained in a time series, the range still can be selected
ts["2011-01-06":"2011-01-11"]

# use truncate slice a series between two dates
ts.truncate(after="2011-01-09")


In [ ]:
times = ts.truncate(after="2011-01-09")
times[:] = np.nan


In [ ]:
ts


In [ ]:
times


In [ ]:
#  Indexing is same for DataFrame

dates = pd.date_range("2000-01-01", periods=100, freq="W-WED")

long_df = pd.DataFrame(
    np.random.standard_normal((100, 4)),
    index=dates,
    columns=["Coloradi", "Texas", "New York", "Ohio"],
)

long_df.loc["2001-05"]


## Time Series with Duplicated Indices

When there are multiple observations failing on a particular timestamp.


In [ ]:
dates = pd.DatetimeIndex(
    ["2000-01-01", "2000-01-02", "2000-01-02", "2000-01-02", "2000-01-03"]
)

dup_ts = pd.Series(np.arange(5), index=dates)


In [ ]:
dup_ts.index.is_unique


In [ ]:
# To aggregate the data having nonunique timestamps

grouped = dup_ts.groupby(level=0)

# To get the dataframe without duplications
grouped.mean()


# 11.3 Date Ranges, Frequencies and Shifting

Generic time series in pandas are assumed to be irregular - no fixed frequencies.

Convert the sample time series to fixed daily frequency buy calling resample.


In [ ]:
resampler = ts.resample("D")

# use resampler.mean() to convert the resampler object to DataFrame
resampler.mean()


## Generating Date Ranges

`pandas.date_range` is responsible for generating a DatetimeIndex with an indicated length according to a particular frequency.


In [ ]:
# By default, pandas.date_range generates daily timestamps.
index = pd.date_range("2012-04-01", "2012-06-01")

# Pass only a start or end date, must pass a number of periods to generate
# generate 20 days timestamps starts from 2012-04-01
pd.date_range(start="2012-04-01", periods=20)

# frequency "BM" stands to BusinessMonthEnd
pd.date_range("2000-01-01", "2000-12-01", freq="BM")

# date_range preserves the time of start and end timestamp
pd.date_range("2012-05-02 12:56:31", periods=5)

# Normalize the timestamp by passing the normalize option
pd.date_range("2012-05-02 12:56:31", periods=5, normalize=True)


## Frequencies and Dare Offsets

Frequencies in pandas are composed of a base frequency and a multiplier.
Base frequencies are refereed to by a string alias. "M" for monthly , 'H' for hourly

For each base frequency there is an object referred to as a date offset.


In [ ]:
from pandas.tseries.offsets import Hour, Minute

hour = Hour()
hour

four_hours = Hour(4)
four_hours

# Putting an integer before the base frequency to create a multiple
pd.date_range("2000-01-01", "2000-01-03 23:59", freq="4H")

# combine offsets by addition
Hour(2) + Minute(30)

pd.date_range("2000-01-01", periods=10, freq="1h30min")


## Week of month dates

Frequency class starting with WOM enables you to get dates like the third Friday of each month


In [ ]:
# get every 3rd Friday of each week from start to end
monthly_dates = pd.date_range("2012-01-01", "2012-09-01", freq="WOM-3FRI")

monthly_dates


## Shifting (Leading and Lagging) Data

Shifting refers to moving data backword and forward through time.

Both Series and DataFrame have a shift method for doing naive shifts forward ot backword, leaving the index unmodified.


In [ ]:
ts = pd.Series(
    np.random.standard_normal(4), index=pd.date_range("2000-01-01", periods=4, freq="M")
)

ts.shift(2)
ts.shift(-2)

# use shift to computing consecutive percent changes in a time series or multiple time series as DataFrame columns
ts / ts.shift(1) - 1

# Naive shifts leave the index unmodified
# If frequency is known, the shift is moving the timestamps instead of the data
ts.shift(2, freq="M")

ts.shift(3, freq="D")

# T stands for minutes.
# the freq parameter indicates the offset to apply to the timestamps, it does not change the underlaying frequency of the data
ts.shift(1, freq="90T")


### Shifting dates with offsets

The pandas date offsets can also be used with datetime or Timestamp objects.


In [ ]:
from pandas.tseries.offsets import Day, MonthEnd

now = datetime(2011, 11, 7)

now + 3 * Day()

# Anchored offsets can "roll" dates forward or backward
offset = MonthEnd()

# rollforward to the nearest month end
offset.rollforward(now)

# rollback now to the nearedt month end
offset.rollback(now)

# Use these methods with groupby
ts = pd.Series(
    np.random.standard_normal(20),
    index=pd.date_range("2000-01-15", periods=20, freq="4D"),
)

# rollforward all time to end of the month and group by index
# calculate the mean() value to return a dataframe
ts.groupby(MonthEnd().rollforward).mean()
# Same as below
ts.resample("M").mean()


# 11.4 Time Zone Handling

Time zones are expressed as offsets from UTC

Python time zone information comes from pytz library.


In [3]:
import pytz

pytz.common_timezones[-5:]

# to get a time zone object from pytz, use pytz.timezone
tz = pytz.timezone("America/New_York")

tz


<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

## Time Zone Localization and Conversion

By default, time series in pandas are time zone naive.


In [17]:
dates = pd.date_range("2012-03-09 09:30", periods=6)

ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)

print(ts.index.tz)  # None by default

# To set a date range with a timezone
pd.date_range("2012-03-09 09:30", periods=10, tz="UTC")

# Use `tz_localize` method to convert naive to localized
ts_utc = ts.tz_localize("UTC")

# Once a time series has been localized to a particular timezone
# it can be converted to another timezone with tz_convert
ts_utc.tz_convert("America/New_York")


None


2012-03-09 04:30:00-05:00   -1.636498
2012-03-10 04:30:00-05:00   -1.301687
2012-03-11 05:30:00-04:00    0.846252
2012-03-12 05:30:00-04:00   -0.349391
2012-03-13 05:30:00-04:00    1.138519
2012-03-14 05:30:00-04:00    2.060117
Freq: D, dtype: float64

## Operations with Time Zone-Aware Timestamp Objects

Individual Timestamp object can be localized from naive to time zone-aware
and converted from one time zone to another.


In [21]:
stamp = pd.Timestamp("2011-03-12 04:00")

stamp_utc = stamp.tz_localize("utc")

stamp_utc.tz_convert("Asia/Shanghai")

# pass time zone argument when create the TimeStamp
stamp_moscow = pd.Timestamp("2011-03-12 04:00", tz="Europe/Moscow")

# change time zone doesn't alter the internal UTC value
stamp_utc.value == stamp_utc.tz_convert("Asia/Singapore").value  # True


True

## Operations Between Different Time Zones

If two time series with different time zone are combined
The result will be UTC.


In [27]:
dates = pd.date_range("2012-03-07 09:30", periods=10, freq='B')

ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)

ts1 = ts[:7].tz_localize("Europe/London")
ts2 = ts1[2:].tz_convert("Europe/Moscow")

result = ts1 + ts2

result


2012-03-07 09:30:00+00:00         NaN
2012-03-08 09:30:00+00:00         NaN
2012-03-09 09:30:00+00:00   -0.693410
2012-03-12 09:30:00+00:00   -0.859745
2012-03-13 09:30:00+00:00    1.760350
2012-03-14 09:30:00+00:00    2.823902
2012-03-15 09:30:00+00:00    0.784757
dtype: float64

# 11.5 Periods and Period Arithmetic

Periods represent time spans (days, months, quarters)

pandas.Period class represents Periods data type, requiring a string or integer and a supported frequency


In [32]:
# A Period object represents the full time span from Jan 1 2011 to dec 31 2011, entire year
p = pd.Period('2011', freq="A-DEC")

# Adding and subtracting integers from periods will shift the period frequency
p + 5

# If two periods have the same frequency, their difference is the number of units between then as a date offset
pd.Period('2014', freq="A-DEC") - p

# Regular ranges of periods can be constructed with the period_range function
periods = pd.period_range("2000-01-01", "2000-06-30", freq="M")
periods

# PeriodIndex class stores a sequence of periods and can serve as an axis index in any pandas data structure. 
pd.Series(np.random.standard_normal(6), index=periods)

2000-01    0.000797
2000-02    0.752481
2000-03    0.400136
2000-04   -0.406112
2000-05   -1.335364
2000-06   -0.833035
Freq: M, dtype: float64

## Period Frequency Conversion

Periods and PeriodIndex can be converted to another frequency with `asfreq` method.

In [14]:
# Convert annual period to monthly period
p = pd.Period('2011', freq='A-DEC')

p.asfreq('M', how="start")

p.asfreq("M", how="end")

# For a fiscal year (financial year) ending on a month other than December
# the corresponding monthly subsperiods are different
p = pd.Period('2011', freq='A-JUN')

# The fiscal year will start on July and end on June
p.asfreq("M", how='start')
p.asfreq("M", how='end')

# Convert PeriodIndex objects
periods = pd.period_range('2006', '2009', freq='A-DEC')

ts = pd.Series(np.random.standard_normal(len(periods)), index = periods)

# Change the frequency of the year to 1st month of the year
ts.asfreq('M', how='start')

# Get the last business day of the year
ts.asfreq('B', how='end')


2006-12-29   -0.272109
2007-12-31   -0.199437
2008-12-31   -0.199964
2009-12-31    1.464593
Freq: B, dtype: float64

## Quarterly Period Frequencies

Much quarterly data is reported relative to a fiscal year end. 

In [32]:
p = pd.Period("2012Q4", freq="Q-JAN")

# Check the start and end of fiscal year by check the daily frequency
p.asfreq("D", how="start")
p.asfreq("D", how="end")

# Get timestamp of 4 p/m. on the second-to-ast business day of the quarter
p4pm = (p.asfreq("B", how="END") - 1).asfreq("T", how="start") + 16 * 60

p4pm

p4pm.to_timestamp().timetz()

# Generate quarterly ranges
periods = pd.period_range("2011Q3", "2012Q4", freq="Q-JAN")

ts = pd.Series(np.arange(len(periods)), index=periods)

# Get the 2nd last business day of each quarter and second hour
new_periods = (periods.asfreq('B', 'end') - 1).asfreq('H', 'start') + 1

ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int64

In [31]:
ts.index = new_periods
ts

2010-10-28 01:00    0
2011-01-28 01:00    1
2011-04-28 01:00    2
2011-07-28 01:00    3
2011-10-28 01:00    4
2012-01-30 01:00    5
Freq: H, dtype: int64

## 